In [57]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from mongoengine import *
from urllib.parse import urlparse, parse_qsl
from gensim.models import Word2Vec, Doc2Vec
connect('mongoengine_documents')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
from monary import Monary
mon = Monary()

from utils.utils import *
from RISparser import readris

from scoping.models import *

from mongo_classes import *
import time
import gensim

In [93]:
def read_docs(fname, tokens_only=False):
    for i, doc in enumerate(docs.iterator()):
        yield gensim.utils.simple_preprocess(doc.title)
        
titles = list(read_docs(docs))
dictionary = gensim.corpora.Dictionary(titles)
corpus = [dictionary.doc2bow(t) for t in titles]
tf_idf = gensim.models.TfidfModel(corpus)

tfidf_sims = gensim.similarities.Similarity('/queries/sims/',tf_idf[corpus],
                                      num_features=len(dictionary))


/home/galm/software/django/tmvenv_36/lib/python3.6/site-packages/gensim/matutils.py:718: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[]

In [77]:
def find_tfidf_similar(r):
    doc_bow = dictionary.doc2bow(gensim.utils.simple_preprocess(r['ti']))
    query_doc_tf_idf = tf_idf[doc_bow]
    return tfidf_sims[query_doc_tf_idf]
     
doc = {"ti":"the community connection model implementation of best evidence into practice for self management of chronic diseases"   }
x = find_tfidf_similar(doc)

ValueError: need at least one array to concatenate

In [87]:
r = doc
doc_bow = dictionary.doc2bow(gensim.utils.simple_preprocess(r['ti']))
query_doc_tf_idf = tf_idf[doc_bow]
query_doc_tf_idf
#tfidf_sims[query_doc_tf_idf]

[]

In [90]:
print(tf_idf)

TfidfModel(num_docs=0, num_nnz=0)


In [62]:
model = Doc2Vec.load("/queries/title_model")

In [65]:

def find_vec_similar(r):
    inferred_vector = model.infer_vector(gensim.utils.simple_preprocess(r['ti']),steps=15)
    sims = model.docvecs.most_similar([inferred_vector], topn=5)
    for s in sims:
        if s[1] > .65:
            return Doc.objects.get(pk=s[0])

In [38]:
def find_jaccard_similar(r):
    s1 = shingle(get(r,'ti'))

    twords = get(r,'ti').split()
    if len(twords) > 1:
        twords = ' '.join([x for x in twords[0:1]])
    else:
        twords = twords[0]

    py_docs = scoping.models.Doc.objects.filter(
        PY=get(r,'py'),
        title__iregex='\w',
        title__icontains=twords
    )
    doc = None
    for d in py_docs.iterator():
        j = jaccard(s1,d.shingle())
        if j > 0.51:
            doc = d
            return d
    return None

In [45]:
t0 = time.time()
matches = []
matched = []
with open("mozart.ris") as f:
    entries = readris(f,mapping=RIS_KEY_MAPPING)
    for i,e in enumerate(entries):
        if i > 10:
            break
        d = find_jaccard_similar(e)
        if d is not None:
            matched.append(d)
            matches.append(e)
            
        
print(f"elapsed: {time.time() - t0:.2}")
print(len(matches))

elapsed: 1.4e+01
2


In [ ]:
print([m.title for m in matched])
print([m['ti'] for m in matches])

['The effect of music therapy on the cardiac activity parameters of a fetus in a cardiotocographic examination', 'Neurosurgery and Music; Effect of Wolfgang Amadeus Mozart']
['The effect of music therapy on the cardiac activity parameters of a fetus in a cardiotocographic examination', 'Neurosurgery and Music; Effect of Wolfgang Amadeus Mozart']


In [66]:
t0 = time.time()
matches = []
matched = []
with open("mozart.ris") as f:
    entries = readris(f,mapping=RIS_KEY_MAPPING)
    for i,e in enumerate(entries):
        if i > 10:
            break
        d = find_vec_similar(e)
        if d is not None:
            matched.append(d)
            matches.append(e)
            
        
print(f"elapsed: {time.time() - t0:.2}")
print(len(matches))

/home/galm/software/django/tmvenv_36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


elapsed: 0.67
6


In [67]:
doc = 

['Thermodynamic estimation of the probability of chemical reactions during alkaline decomposition of eudialyte concentrate', 'Strigolactones are required for nitric oxide to induce root elongation in response to nitrogen and phosphate deficiencies in rice', 'MEMBERS REPORT SMALLER (3-PERCENT) AVERAGE SALARY GAIN', 'Unraveling the Skilled Mobility for Sustainable Development Mantra: An Analysis of China-EU Academic Mobility', 'Operation of a solid oxide fuel cell on a reformed FAME mixture', 'A GIS-based analysis on the relationship between the annual available amount and the procurement cost of forest biomass in a mountainous region in Japan']
['Amadeus: A vision of musical genius', 'The genre of the instrumental concerto in the viola repertoire: From bach to schnittke', 'The effect of music therapy on the cardiac activity parameters of a fetus in a cardiotocographic examination', 'Neurosurgery and Music; Effect of Wolfgang Amadeus Mozart', 'MICHAEL FINNISSY and WOLFGANG AMADEUS MOZART